# Generate a real estate financial model using python

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("/home/greg/code/gregso10/financial_models/real_estate/data/ValeursFoncieres-2025-S1.txt", sep= "|")

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df[["No voie","Type de voie","Voie","Code postal","Commune","Valeur fonciere"]]

In [ ]:
df.head(5)

In [ ]:
df["Commune"].unique()

In [ ]:
reduced_df = df[df["Commune"] == "PARIS 11"]

### Create a reduced DF which contains
1. location
2. price
3. type
4. size
5. surface réelle batie
6. surface terrain
7. nature mutation

Problèmes à résoudre: 
- identifier et rattacher les bonnes parcelles aux bonnes ventes
--> certaines ventes sont sur plusieurs parcelles et composent plusieurs lots...

In [ ]:
reduced_df

In [ ]:
import requests
import time

In [ ]:
# 2. Création d'un identifiant unique par transaction (Mutation)
# Une mutation est définie par sa date, son numéro de disposition et sa valeur.
reduced_df['id_mutation'] = (reduced_df['Date mutation'].astype(str) + '_' + 
                    reduced_df['Valeur fonciere'].astype(str) + '_' + 
                    reduced_df['No disposition'].astype(str) + '_' +
                    reduced_df['Code commune'].astype(str))

# 3. Filtrer uniquement les ventes (on exclut les échanges, adjudications, etc.)
reduced_df = reduced_df[reduced_df['Nature mutation'] == 'Vente']

# 4. Analyse des lots par transaction
# On veut savoir combien de maisons et d'appartements il y a dans CHAQUE vente.
def analyze_mutation(group):
    # Compte des types de locaux
    n_maisons = len(group[group['Type local'] == 'Maison'])
    n_apparts = len(group[group['Type local'] == 'Appartement'])
    n_dep = len(group[group['Type local'] == 'Dépendance'])
    
    # Surface totale habitable (uniquement maisons et apparts)
    surface_reelle = group[group['Type local'].isin(['Maison', 'Appartement'])]['Surface reelle bati'].sum()
    
    # Récupération des infos uniques (Prix, Date, Adresse)
    first_row = group.iloc[0]
    
    return pd.Series({
        'valeur_fonciere': first_row['Valeur fonciere'],
        'date': first_row['Date mutation'],
        'commune': first_row['Commune'],
        'n_maisons': n_maisons,
        'n_apparts': n_apparts,
        'n_dependances': n_dep,
        'surface_habitable': surface_reelle,
        'latitude': 48.85, # Simulé ici, à récupérer via géocodage
        'longitude': 2.35  # Simulé ici
    })

# On regroupe par transaction
df_transactions = reduced_df.groupby('id_mutation').apply(analyze_mutation).reset_index()

# 5. Conversion du prix en numérique (le format français utilise la virgule)
df_transactions['valeur_fonciere'] = df_transactions['valeur_fonciere'].str.replace(',', '.').astype(float)

# ---------------------------------------------------------
# FILTRAGE INTELLIGENT POUR LE MACHINE LEARNING
# ---------------------------------------------------------

# On garde uniquement les ventes "unimates" (1 seule unité d'habitation)
# C'est-à-dire : (1 Maison et 0 Appart) OU (0 Maison et 1 Appart)
df_ml = df_transactions[
    ((df_transactions['n_maisons'] == 1) & (df_transactions['n_apparts'] == 0)) |
    ((df_transactions['n_maisons'] == 0) & (df_transactions['n_apparts'] == 1))
].copy()

# 6. Calcul du Prix au m² (Variable cible clé)
df_ml['prix_m2'] = df_ml['valeur_fonciere'] / df_ml['surface_habitable']

# 7. Nettoyage final (retirer les surfaces nulles ou aberrantes)
df_ml = df_ml[df_ml['surface_habitable'] > 9] # Exclure les "micro-lots" ou erreurs

print(f"Transactions brutes : {len(df_transactions)}")
print(f"Transactions propres pour ML : {len(df_ml)}")
df_transactions.head()

In [ ]:
import pydeck as pdk
import requests
import time

In [ ]:
# --- 1. Fonction de Nettoyage ---
def clean_dvf_data(df):
    df['id_mutation'] = (df['Date mutation'].astype(str) + '_' + 
                         df['Valeur fonciere'].astype(str) + '_' + 
                         df['No disposition'].astype(str) + '_' + 
                         df['Code commune'].astype(str))

    def analyze_mutation(group):
        first = group.iloc[0]
        # Construction adresse
        num = str(int(first['No voie'])) if pd.notna(first['No voie']) else ""
        type_voie = str(first['Type de voie']) if pd.notna(first['Type de voie']) else ""
        voie = str(first['Voie']) if pd.notna(first['Voie']) else ""
        code_postal = str(int(first['Code postal'])) if pd.notna(first['Code postal']) else ""
        commune = str(first['Commune']) if pd.notna(first['Commune']) else ""
        full_address = f"{num} {type_voie} {voie} {code_postal} {commune}".strip()
        
        return pd.Series({
            'valeur_fonciere': float(str(first['Valeur fonciere']).replace(',', '.')),
            'adresse_complete': full_address,
            'surface_totale': group['Surface reelle bati'].sum()
        })

    # Filtrage Vente uniquement
    return df[df['Nature mutation'] == 'Vente'].groupby('id_mutation').apply(analyze_mutation).reset_index()

# --- 2. Chargement et Nettoyage ---
try:
    # Remplace par le chemin réel de ton fichier si nécessaire
    df_raw = reduced_df 
    df_clean = clean_dvf_data(df_raw)
    print(f"Données nettoyées : {len(df_clean)} transactions uniques trouvées.")
except FileNotFoundError:
    print("Erreur : Fichier introuvable. Vérifie le chemin.")

# --- 3. Géocodage (Avec barre de progression) ---
print("Démarrage du géocodage...")
lats, lons = [], []

for i, address in enumerate(df_clean['adresse_complete']):
    if i % 5 == 0: # Affiche la progression tous les 5 items
        print(f"Traitement : {i}/{len(df_clean)}", end="\r")
        
    if not address or len(address) < 5:
        lats.append(None); lons.append(None)
        continue
        
    try:
        url = "https://api-adresse.data.gouv.fr/search/"
        response = requests.get(url, params={'q': address, 'limit': 1})
        if response.json()['features']:
            coords = response.json()['features'][0]['geometry']['coordinates']
            lons.append(coords[0])
            lats.append(coords[1])
        else:
            lats.append(None); lons.append(None)
    except:
        lats.append(None); lons.append(None)
    time.sleep(0.05) # Respect de l'API

df_clean['latitude'] = lats
df_clean['longitude'] = lons
df_geo = df_clean.dropna(subset=['latitude', 'longitude'])

print(f"\nGéocodage terminé : {len(df_geo)} adresses localisées.")
df_geo.head()

In [ ]:
# --- 1. Fonction de Nettoyage ---
def clean_dvf_data(df):
    df['id_mutation'] = (df['Date mutation'].astype(str) + '_' + 
                         df['Valeur fonciere'].astype(str) + '_' + 
                         df['No disposition'].astype(str) + '_' + 
                         df['Code commune'].astype(str))

    def analyze_mutation(group):
        first = group.iloc[0]
        # Construction adresse
        num = str(int(first['No voie'])) if pd.notna(first['No voie']) else ""
        type_voie = str(first['Type de voie']) if pd.notna(first['Type de voie']) else ""
        voie = str(first['Voie']) if pd.notna(first['Voie']) else ""
        code_postal = str(int(first['Code postal'])) if pd.notna(first['Code postal']) else ""
        commune = str(first['Commune']) if pd.notna(first['Commune']) else ""
        full_address = f"{num} {type_voie} {voie} {code_postal} {commune}".strip()
        
        return pd.Series({
            'valeur_fonciere': float(str(first['Valeur fonciere']).replace(',', '.')),
            'adresse_complete': full_address,
            'surface_totale': group['Surface reelle bati'].sum()
        })

    # Filtrage Vente uniquement
    return df[df['Nature mutation'] == 'Vente'].groupby('id_mutation').apply(analyze_mutation).reset_index()

In [ ]:
def get_gps_coordinates(address):
    """
    Interroge l'API Adresse (BAN) pour une adresse donnée.
    Renvoie un tuple (latitude, longitude) ou (None, None) si échec.
    """
    if not address or len(str(address)) < 5:
        return None, None

    API_URL = "https://api-adresse.data.gouv.fr/search/"
    params = {
        'q': address,
        'limit': 1 # On veut juste le meilleur résultat
    }
    
    try:
        response = requests.get(API_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            if data['features']:
                # L'API renvoie [long, lat], on veut (lat, long)
                coords = data['features'][0]['geometry']['coordinates']
                return coords[1], coords[0] 
    except Exception as e:
        print(f"Erreur connexion pour {address}: {e}")
        
    return None, None

In [ ]:
print("Début du géocodage...")

# On crée deux listes vides
lats = []
lons = []

# On boucle sur chaque ligne du DataFrame nettoyé
for index, row in df_clean.iterrows():
    lat, lon = get_gps_coordinates(row['adresse_complete'])
    lats.append(lat)
    lons.append(lon)
    
    # Petite pause pour être poli avec l'API (facultatif si peu de données)
    # time.sleep(0.02) 

# On assigne les colonnes d'un coup
df_clean['latitude'] = lats
df_clean['longitude'] = lons

# On retire les échecs (adresses introuvables)
df_geo = df_clean.dropna(subset=['latitude', 'longitude'])

print(f"Géocodage terminé ! {len(df_geo)} adresses localisées sur {len(df_clean)}.")
df_geo.head()

In [ ]:
# Configuration des couleurs (Logique de gradient simple)
max_price = df_geo['valeur_fonciere'].max()
df_geo['color'] = df_geo['valeur_fonciere'].apply(
    lambda x: [int((x/max_price)*255), int(255-(x/max_price)*255), 50, 160]
)

# --- Configuration de la Carte ---
view_state = pdk.ViewState(
    latitude=df_geo['latitude'].mean(),
    longitude=df_geo['longitude'].mean(),
    zoom=13,
    pitch=50, # Inclinaison pour l'effet 3D
    bearing=10
)

column_layer = pdk.Layer(
    "ColumnLayer",
    data=df_geo,
    get_position=["longitude", "latitude"],
    get_elevation="valeur_fonciere", # Hauteur = Prix
    elevation_scale=0.05,            # Échelle à ajuster selon tes montants (0.01 à 0.1)
    radius=30,                       # Largeur des colonnes en mètres
    get_fill_color="color",
    pickable=True,
    auto_highlight=True,
    extruded=True
)

# Tooltip (Info-bulle au survol)
tooltip = {
    "html": "<b>Adresse:</b> {adresse_complete}<br/><b>Prix:</b> {valeur_fonciere} €",
    "style": {"background": "grey", "color": "white", "font-family": "Arial", "z-index": "1000"}
}

# --- Affichage ---
r = pdk.Deck(
    layers=[column_layer],
    initial_view_state=view_state,
    tooltip=tooltip,
    map_style="mapbox://styles/mapbox/light-v9"
)

# Cette commande affiche la carte DANS le notebook
r.show

In [ ]:
"""
GeoJsonLayer
===========

Property values in Vancouver, Canada, adapted from the deck.gl example pages. Input data is in a GeoJSON format.
"""

import pydeck as pdk

DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

INITIAL_VIEW_STATE = pdk.ViewState(latitude=49.254, longitude=-123.13, zoom=11, max_zoom=16, pitch=45, bearing=0)

polygon = pdk.Layer(
    "PolygonLayer",
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 20],
)

geojson = pdk.Layer(
    "GeoJsonLayer",
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation="properties.valuePerSqm / 20",
    get_fill_color="[255, 255, properties.growth * 255]",
    get_line_color=[255, 255, 255],
)

r = pdk.Deck(layers=[polygon, geojson], initial_view_state=INITIAL_VIEW_STATE)

r.to_html("geojson_layer.html")